# Lab 12-1 hello rnn

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pprint import pprint
tf.set_random_seed(777)

### Prepairing dataset
hihell -> ihello, hell -> ello

In [2]:
strings = ['hihello', 'hello']
char_set = ['<pad>'] + sorted(list(set('hihello' + 'hello')))
idx2char = {idx : char for idx, char in enumerate(char_set)}
char2idx = {char : idx for idx, char in enumerate(char_set)}

x_strings = ['hihell', 'hell']
y_strings = ['ihello', 'ello']

# idx2char, char2idx 각각의 dictionary 확인
print(idx2char)
print(char2idx)

{0: '<pad>', 1: 'e', 2: 'h', 3: 'i', 4: 'l', 5: 'o'}
{'<pad>': 0, 'e': 1, 'h': 2, 'i': 3, 'l': 4, 'o': 5}


### Preprocessing dataset

In [3]:
x_data = list(map(lambda strings : [char2idx.get(char) for char in strings], x_strings))
y_data = list(map(lambda strings : [char2idx.get(char) for char in strings], x_strings))
x_data_len = list(map(lambda strings : len(strings), x_data))

# padding
max_length = 8
x_data = pad_sequences(sequences = x_data, maxlen = max_length, padding = 'post', truncating = 'post')
y_data = pad_sequences(sequences = y_data, maxlen = max_length, padding = 'post', truncating = 'post')

# data 형태확인
print(x_data)
print(y_data)
print(x_data_len)

[[2 3 2 1 4 4 0 0]
 [2 1 4 4 0 0 0 0]]
[[2 3 2 1 4 4 0 0]
 [2 1 4 4 0 0 0 0]]
[6, 4]


### Creating graph

In [4]:
# input
data = tf.data.Dataset.from_tensor_slices((x_data, y_data, x_data_len))
data = data.shuffle(buffer_size = 2)
data = data.batch(batch_size = 1)
iterator = data.make_initializable_iterator()
x, y, x_len = iterator.get_next()

# hyper-parameters for lstm (many to many), one-hot encoding
n_of_classes = len(idx2char)
hidden_size = 10

one_hot_encoding = tf.eye(num_rows = n_of_classes, dtype = tf.float32)
one_hot_encoding = tf.get_variable(name = 'one_hot_encoding',
                                   initializer = one_hot_encoding, trainable = False)

x_batch = tf.nn.embedding_lookup(params = one_hot_encoding, ids = x)
lstm_cell = tf.nn.rnn_cell.LSTMCell(num_units = hidden_size, dtype = tf.float32)
score_cell = tf.contrib.rnn.OutputProjectionWrapper(cell = lstm_cell, output_size = n_of_classes)
outputs, _ = tf.nn.dynamic_rnn(cell = score_cell, inputs = x_batch,
                               sequence_length = x_len, dtype = tf.float32)

masking = tf.sequence_mask(lengths = x_len, maxlen = max_length, dtype = tf.float32)
s2s_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = y, weights = masking)

prediction = tf.argmax(input = outputs, axis = -1)

# training
lr = .1
opt = tf.train.AdamOptimizer(learning_rate = lr)
training_op = opt.minimize(loss = s2s_loss)

### Training

In [5]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config = sess_config)
sess.run(tf.global_variables_initializer())

In [6]:
epochs = 20
tr_loss_hist = []

for epoch in range(epochs):
    
    sess.run(iterator.initializer)
    avg_tr_loss = 0
    step = 0
    
    try:
        while True:
            _, loss = sess.run([training_op, s2s_loss])
            avg_tr_loss += loss
            step += 1
    except:
        avg_tr_loss /= step
        tr_loss_hist.append(avg_tr_loss)
        
    print('epoch : {:3}, loss : {:.3f}'.format(epoch + 1, avg_tr_loss))

epoch :   1, loss : 1.619
epoch :   2, loss : 1.086
epoch :   3, loss : 0.796
epoch :   4, loss : 0.462
epoch :   5, loss : 0.278
epoch :   6, loss : 0.144
epoch :   7, loss : 0.072
epoch :   8, loss : 0.040
epoch :   9, loss : 0.022
epoch :  10, loss : 0.012
epoch :  11, loss : 0.007
epoch :  12, loss : 0.004
epoch :  13, loss : 0.003
epoch :  14, loss : 0.002
epoch :  15, loss : 0.002
epoch :  16, loss : 0.001
epoch :  17, loss : 0.001
epoch :  18, loss : 0.001
epoch :  19, loss : 0.001
epoch :  20, loss : 0.001


### Accuracy

In [7]:
yhat = sess.run(prediction, feed_dict = {x : x_data, x_len : x_data_len})

In [8]:
print('true : {}, prediction : {}'.format(y_data, yhat))

true : [[2 3 2 1 4 4 0 0]
 [2 1 4 4 0 0 0 0]], prediction : [[2 3 2 1 4 4 0 0]
 [2 1 4 4 0 0 0 0]]
